# Data Acquisition

---

This notebook outlines the process for acquiring the Zillow property data needed for this project. This notebook also demonstrates the steps to create and test the functions in the util/acquire.py file.

---

## Accessing the Database

The Zillow property data is located in the MySQL database at data.codeup.com in the zillow database. In order to access this data you will need login credentials. Assuming you have credentials save these in a env.py file in the following form:

In [1]:
username = 'your_username'
password = 'your_password'
hostname = 'data.codeup.com'

Save this file in the notebooks directory (also save a copy in the main directory for use with the final report notebook). Once we have our login credentials we can import them from env and create our database URL.

In [2]:
from env import username, password, hostname
database_name = 'zillow'

# This is the template for a database URL, we simply plug in our login credentials and the database we want to read from.
url = f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}'

For our convenience we will turn this into a function.

In [3]:
def get_db_url(database_name, username = username, password = password, hostname = hostname):
    return f'mysql+pymysql://{username}:{password}@{hostname}/{database_name}'

---

## Acquiring the Data

The database contains several tables. The data we need is from the properties_2017 table. This table has a large number of columns including a few columns that we are not allowed to use so we will only select the columns that we will be analyzing. Additionally we need to make sure we're only selecting properties that are single family residential properties. Finally, we only want properties that had a transaction in 2017. The SQL query below grabs all the data we need:

In [4]:
sql = """
SELECT
    bedroomcnt,
    bathroomcnt,
    calculatedfinishedsquarefeet,
    taxvaluedollarcnt,
    yearbuilt,
    fips,
    numberofstories,
    basementsqft,
    fireplacecnt,
    heatingorsystemdesc,
    roomcnt,
    garagetotalsqft,
    hashottuborspa,
    poolcnt,
    poolsizesum,
    yardbuildingsqft17
FROM properties_2017
JOIN propertylandusetype
    ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
    AND (propertylandusetype.propertylandusedesc IN ('Single Family Residential', 'Inferred Single Family Residential'))
LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid);
"""

We can create a function to return this query for us.

In [5]:
def get_zillow_sql():
    return """
        SELECT
            bedroomcnt,
            bathroomcnt,
            calculatedfinishedsquarefeet,
            taxvaluedollarcnt,
            yearbuilt,
            fips,
            numberofstories,
            basementsqft,
            fireplacecnt,
            heatingorsystemdesc,
            roomcnt,
            garagetotalsqft,
            hashottuborspa,
            poolcnt,
            poolsizesum,
            yardbuildingsqft17
        FROM properties_2017
        JOIN propertylandusetype
            ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
            AND (propertylandusetype.propertylandusedesc IN ('Single Family Residential', 'Inferred Single Family Residential'))
        LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid);
        """

Now we can read the data from the MySQL database. We'll need pandas for this.

In [6]:
import pandas as pd

In [7]:
properties = pd.read_sql(sql, get_db_url('zillow'))
properties.shape

(2985416, 13)

For our convenience we can put all of this into a function which will check if the .csv file exists and read from there if it does, otherwise read from the MySQL database and cache the data for quicker access. We will also include a parameter use_cache
that will indicate whether or not to use the .csv file if it is available in case we would like to pull the data from the database regardless if the .csv file exists.

In [8]:
import os

def get_zillow_data(use_cache = True):
    # If the file is cached, read from the .csv file
    if os.path.exists('zillow.csv') and use_cache:
        print('Using cache')
        return pd.read_csv('zillow.csv')
    
    # Otherwise read from the mysql database
    else:
        print('Reading from database')
        df = pd.read_sql(get_zillow_sql(), get_db_url('zillow'))
        df.to_csv('zillow.csv', index = False)
        return df

In [9]:
get_zillow_data().head(2)

Reading from database


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips,numberofstories,basementsqft,fireplacecnt,typeconstructiondesc,heatingorsystemdesc,architecturalstyledesc,propertylandusedesc
0,4.0,2.0,1848.0,64712.0,1952.0,6111.0,1.0,NaN,1.0,None,None,None,Single Family Residential
1,2.0,2.0,1430.0,381638.0,1952.0,6037.0,NaN,NaN,NaN,None,Central,None,Single Family Residential


In [10]:
get_zillow_data().head(2)

Using cache


,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips,numberofstories,basementsqft,fireplacecnt,typeconstructiondesc,heatingorsystemdesc,architecturalstyledesc,propertylandusedesc
0,4.0,2.0,1848.0,64712.0,1952.0,6111.0,1.0,NaN,1.0,NaN,NaN,NaN,Single Family Residential
1,2.0,2.0,1430.0,381638.0,1952.0,6037.0,NaN,NaN,NaN,NaN,Central,NaN,Single Family Residential


Everything seems to be working correctly.

---

## Conclusion

Now we can easily acquire our data. These functions will be located in the acquire.py file in the util directory for use with the final report.